# Magenetic monopole and spallation event+statistics display

In [1]:
from bokeh.io import output_notebook
from RootInteractive.Tools.aliTreePlayer import *
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from RootInteractive.Tools.aliTreePlayer import *
from bokeh.io import curdoc
import numpy as np
import os
import sys
from ROOT import TFile, gSystem
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from RootInteractive.MLpipeline.NDFunctionInterface import  *
import plotly.express as px
#output_notebook()
from IPython.display import Image
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import matplotlib.pyplot as plt
from bokeh.transform import factor_cmap, factor_mark

from scipy.stats import norm
import matplotlib.mlab as mlab
import scipy.stats as stats
import math
from RootInteractive.Tools.compressArray import *
from pprint import pprint

Welcome to JupyROOT 6.20/08
x bokehVisJS3DGraph.ts
x HistogramCDS.ts
x Histo2dCDS.ts


Using TensorFlow backend.
/home/stephan/alice/sw/ubuntu1804_x86-64/Python-modules/1.0-3/share/python-modules/lib/python/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/stephan/alice/sw/ubuntu1804_x86-64/Python-modules/1.0-3/share/python-modules/lib/python/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/stephan/alice/sw/ubuntu1804_x86-64/Python-modules/1.0-3/share/python-modules/lib/python/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future versio

In [2]:
%%time
ROOT.gROOT.LoadMacro("/home/stephan/QC/alice-tpc-notes/JIRA/ATO-432/code/monoAnalysis.C+g")
ROOT.AliDrawStyle.SetDefaults()
ROOT.AliDrawStyle.ApplyStyle("figTemplate")
c=ROOT.TCanvas()
ROOT.gStyle.SetOptStat(0)

CPU times: user 807 ms, sys: 184 ms, total: 991 ms
Wall time: 1 s


Warning in <TClassTable::Add>: class ROOT::VecOps::RVec<float> already in TClassTable
Info in <AliDrawStyle::ApplyStyle>: figTemplate


## Define data compression precission
* 12 bits for position (g.*|r.*)"
* 7 bits for everithing

In [3]:
arrayCompressionRelativeDisplay=[("(g.*|r.*)",[("relative",12), ("code",0), ("zip",0), ("base64",0)]), (".*",[("relative",7), ("code",0), ("zip",0), ("base64",0)])]

In [4]:
#ROOT.loadSeeds()

In [5]:
"""ROOT.treeSeeds1.SetAlias("sector","seed.fSeedIter1.fData.fDetector%36")
ROOT.treeSeeds1.SetAlias("phiSeed","seed.fSeedIter1.fData.fY")
ROOT.treeSeeds1.SetAlias("seed1Tot","seed.fSeedIter1Tot.fData")
ROOT.treeSeeds1.GetListOfAliases().ls()
pprint(ROOT.treeSeeds1.GetEntries())"""

'ROOT.treeSeeds1.SetAlias("sector","seed.fSeedIter1.fData.fDetector%36")\nROOT.treeSeeds1.SetAlias("phiSeed","seed.fSeedIter1.fData.fY")\nROOT.treeSeeds1.SetAlias("seed1Tot","seed.fSeedIter1Tot.fData")\nROOT.treeSeeds1.GetListOfAliases().ls()\npprint(ROOT.treeSeeds1.GetEntries())'

In [21]:
def SpoofDF():
    columns = ["gxSeed","gySeed","chi2N",'fQMeanSeed1',"fQMedianSeed1", "gzSeed","rSeed","eventID","qSeed","sector","phiSeed","seed1Tot"]
    def GetValue(c, j):
        if c=="eventID":
            return j
        else:
            return np.random.randint(20)+1
    arr = np.array([[GetValue(columns[j], i) for j in range(len(columns))] for i in range(100000)])
    arr = np.ascontiguousarray(arr)
    df = pd.DataFrame(arr, columns=columns)
    return df
arrayCompressionRelative16=[(".*",[("relative",16), ("code",0), ("zip",0), ("base64",0)])]  

In [23]:
#%%time
dfSelection="fQMeanSeed1>200"
dfTrack=SpoofDF()#tree2Panda(ROOT.treeSeeds1,["gxSeed","gySeed","chi2N",'fQMeanSeed1',"fQMedianSeed1", "gzSeed","rSeed","eventID","qSeed","sector","phiSeed","seed1Tot"],dfSelection,exclude=[".*XXX"],columnMask=[["Dist_meanG",""]], nEntries=100000)
dfTrack["fQSeed1Ratio"]=dfTrack["fQMeanSeed1"]/dfTrack["fQMedianSeed1"]
dfTrack["fQRatio"]=dfTrack["qSeed"]/dfTrack["fQMedianSeed1"]
print(dfTrack.isnull().values.any())

False


In [8]:
print(dfTrack.shape)
dfTrack.head()

(100000, 14)


,gxSeed,gySeed,chi2N,fQMeanSeed1,fQMedianSeed1,gzSeed,rSeed,eventID,qSeed,sector,phiSeed,seed1Tot,fQSeed1Ratio,fQRatio
0,1,16,5,3,10,14,19,0,6,19,18,9,0.300000,0.6
1,18,2,8,13,1,12,10,1,10,17,1,18,13.000000,10.0
2,0,9,9,10,17,19,15,2,17,13,12,17,0.588235,1.0
3,16,7,2,7,18,18,6,3,18,14,1,13,0.388889,1.0
4,12,18,19,7,0,7,16,4,5,10,15,6,inf,inf


## Event display
* r phi
* r z
* histograms
12 bits data compression (see arrayCompressionRelativeDisplay) for position 7 bits for QA variables

In [9]:
widgetParamsD=[
    ['range', ["sector"]],
    ['range', ['rSeed']],
    ['range', ['phiSeed']],
    #
    ["range", ["gxSeed"]],
    ["range", ["gySeed"]],
    ['range', ['gzSeed']],
    #
    ['range', ['chi2N']],
    ['range', ['fQSeed1Ratio']],
    ['range', ['fQRatio']],
    #
    ['range', ['fQMeanSeed1']],
    ['range', ['fQMedianSeed1']],
    ['range', ['qSeed']],
    #
    ['range', ['seed1Tot']],
    ['range', ['eventID']],
]
widgetLayoutDescD=[ 
    [0,1, 2],     # cylindrical selection
    [3,4,5],      # cartezina selection
    [6,7,8],      # qratio selection 
    [9,10,11],    # q  selection
    [12,13],       # event and Q tot selection
    {'sizing_mode':'scale_width'} 
]

In [24]:
defaultCutTrack="entries>0"
output_file("seed1DisplayRZPhi.html")
histoArray = [
    {"name": "his_chi2N", "variables": ["chi2N"],"nbins":50},
    {"name": "his_fQMeanSeed1", "variables": ["fQMeanSeed1"],"nbins":50},
    {"name": "his_fQMedianSeed1", "variables": ["fQMedianSeed1"],"nbins":50},
    {"name": "his_fQSeed1Ratio", "variables": ["fQSeed1Ratio"],"nbins":50},
]

#dfQA=dfQA.sample(100000)
figureArray = [
    [['rSeed'], ["phiSeed"],  {"colorZvar":"qSeed"}],
    [['rSeed'], ["gzSeed"],  {"colorZvar": "qSeed"}],
    [['chi2N'],["his_chi2N"]],
    [['fQSeed1Ratio'],["his_fQSeed1Ratio"]],
    [['fQMeanSeed1'],["his_fQMeanSeed1"]],
    [['fQMedianSeed1'],["his_fQMedianSeed1"]],
    ["table", {"rowwise": True}],
    {"size": 5}
]    
widgetParams=[
    ['range', ["sector"]],
    ['range', ['rSeed']],
    ['range', ['phiSeed']],
    ['range', ['gzSeed']],
    #
    ['range', ['chi2N']],
    ['range', ['fQSeed1Ratio']],
    ['range', ['fQRatio']],
    ['range', ['fQMeanSeed1']],
    ['range', ['fQMedianSeed1']],
    #
    ['range', ['qSeed']],
    ['range', ['seed1Tot']],
    ['range', ['eventID']],
]
tooltips = [("qSeed", "@qSeed"), ("fQMeanSeed1", "@fQMeanSeed1"), ("fQMedianSeed1", "@fQMedianSeed1"), ("eventID","@eventID"), ("sector","@sector"), ("rSeed","@rSeed")]
widgetLayoutDesc=[ 
    [0,1, 2,3],
    [4, 5,6,7,8],
    [9,10,11], 
    {'sizing_mode':'scale_width'} 
]
figureLayoutDesc=[
    [0,1,{'plot_height':450}],
    [2,3,4,5,{'plot_height':200}],
    [6,{'plot_height':25}],
    {'plot_height':240,'sizing_mode':'scale_width',"legend_visible":False}
]
dfTrack = dfTrack.fillna(1)
fig=bokehDrawSA.fromArray(dfTrack.query("eventID>=0"), "chi2N>0&rSeed>0", figureArray, widgetParamsD,layout=figureLayoutDesc,tooltips=tooltips,sizing_mode='scale_width', widgetLayout=widgetLayoutDescD,nPointRender=3000,rescaleColorMapper=True,arrayCompression=arrayCompressionRelative16,histogramArray=histoArray)

/home/stephan/alice/sw/ubuntu1804_x86-64/Python-modules/1.0-3/share/python-modules/lib/python/site-packages/bokeh/models/plots.py:766: UserWarning:


You are attempting to set `plot.legend.click_policy` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.


/home/stephan/alice/sw/ubuntu1804_x86-64/Python-modules/1.0-3/share/python-modules/lib/python/site-packages/bokeh/models/plots.py:766: UserWarning:


You are attempting to set `plot.legend.visible` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.




True

In [26]:
defaultCutTrack="entries>0"
output_file("seed1DisplayXYRZPhi.html")
histoArray = [
    {"name": "his_chi2N", "variables": ["chi2N"],"nbins":50},
    {"name": "his_fQMeanSeed1", "variables": ["fQMeanSeed1"],"nbins":50},
    {"name": "his_fQMedianSeed1", "variables": ["fQMedianSeed1"],"nbins":50},
    {"name": "his_fQSeed1Ratio", "variables": ["fQSeed1Ratio"],"nbins":50},
]

#dfQA=dfQA.sample(100000)
figureArray = [
    [['gxSeed'], ["gySeed"],  {"colorZvar":"qSeed"}],
    [['rSeed'], ["phiSeed"],  {"colorZvar":"qSeed"}],
    [['rSeed'], ["gzSeed"],  {"colorZvar": "qSeed"}],
    [['chi2N'],["his_chi2N"]],
    [['fQSeed1Ratio'],["his_fQSeed1Ratio"]],
    [['fQMeanSeed1'],["his_fQMeanSeed1"]],
    [['fQMedianSeed1'],["his_fQMedianSeed1"]],
    ["tableHisto", {"rowwise": True}],
    {"size": 5}
]    
widgetParams=[
    ['range', ["sector"]],
    ['range', ['rSeed']],
    ['range', ['phiSeed']],
    ["range", ["gxSeed"]],
     ["range", ["gySeed"]],
    ['range', ['gzSeed']],
    #
    ['range', ['chi2N']],
    ['range', ['fQSeed1Ratio']],
    ['range', ['fQRatio']],
    ['range', ['fQMeanSeed1']],
    ['range', ['fQMedianSeed1']],
    #
    ['range', ['qSeed']],
    ['range', ['seed1Tot']],
    ['range', ['eventID']],
]
tooltips = [("qSeed", "@qSeed"), ("fQMeanSeed1", "@fQMeanSeed1"), ("fQMedianSeed1", "@fQMedianSeed1"), ("eventID","@eventID"), ("sector","@sector"), ("rSeed","@rSeed")]
widgetLayoutDesc=[ 
    [0,1, 2,3],
    [4, 5,6,7,8],
    [9,10,11], 
    {'sizing_mode':'scale_width'} 
]
figureLayoutDesc=[
    [0,1,2,{'plot_height':450}],
    [3,4,5,6,{'plot_height':200}],
    [7,{'plot_height':30}],
    {'plot_height':240,'sizing_mode':'scale_width',"legend_visible":False}
]
fig=bokehDrawSA.fromArray(dfTrack.query("eventID>=0"), "chi2N>0&rSeed>0", figureArray, widgetParamsD,layout=figureLayoutDesc,tooltips=tooltips,sizing_mode='scale_width',
                          widgetLayout=widgetLayoutDescD,nPointRender=6000,rescaleColorMapper=True,arrayCompression=arrayCompressionRelative16,histogramArray=histoArray)

KeyError: 0

In [ ]:
#Display 